In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 기본 directory 설정
import os
os.chdir('/content/drive/MyDrive/Monthly_Workout')

In [1]:
# 모듈 불러오기
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from math import pi

In [22]:
# 데이터 불러오기
path = './' # 기본 directory 경로에 추가 할 경로

train = pd.read_csv(path + 'train_features.csv')
train_labels = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'test_features.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [3]:
act_list=train.iloc[:,2:].columns
acc_list=['acc_x','acc_y','acc_z']
gy_list=['gy_x','gy_y','gy_z']
act_list

Index(['acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z'], dtype='object')

In [4]:
# acc 데이터와 gy 데이터로 분할
def sensor_split(data):
    X_acc = []
    X_gy = []

    for i in tqdm(data['id'].unique()):
        temp_acc = np.array(data[data['id'] == i].loc[:,acc_list])
        temp_gy = np.array(data[data['id'] == i].loc[:,gy_list])
        X_acc.append(temp_acc)
        X_gy.append(temp_gy)
      
    X_acc = np.array(X_acc).reshape(-1,600,3)
    X_gy = np.array(X_gy).reshape(-1,600,3)

    return X_acc, X_gy

In [5]:
# 데이터 증강

def permutation(X, nPerm=4, minSegLength=10):
    X_new = np.zeros(X.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile == True:
        segs = np.zeros(nPerm+1, dtype=int)
        segs[1:-1] = np.sort(np.random.randint(minSegLength, X.shape[0]-minSegLength, nPerm-1))
        segs[-1] = X.shape[0]
        if np.min(segs[1:]-segs[0:-1]) > minSegLength:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        x_temp = X[segs[idx[ii]]:segs[idx[ii]+1],:]
        X_new[pp:pp+len(x_temp),:] = x_temp
        pp += len(x_temp)
    return (X_new)

def aug(data, uid, shift):
    shift_data = np.roll(data[uid], shift, axis=0)
    return shift_data

def rolling(data):
    aug_data=[]
    for i in range(data.shape[0]):
        temp=list((aug(data,i,int(random.random()*600))))
        aug_data.append(temp)
    return np.array(aug_data)

In [5]:
# 데이터 증강 (반복하고 싶은 만큼 조정)
def start_augmentation(train, train_labels):
    # acc, gy 데이터 분할
    X_train_mod=pd.merge(train,train_labels,how='left',on='id')
    X_train_acc, X_train_gy= sensor_split(X_train_mod)

    # 증강시키고 추가할 임시 데이터 복사본
    X_train_acc_temp = X_train_acc.copy()
    X_train_gy_temp = X_train_gy.copy()

    # label 데이터 변환
    y_train = train_labels['label']
    y_train_total = np.append(y_train, y_train, axis=0)

    rep = 3 # 5이상의 경우 reshape 과정에서 reset될 가능성 높음
    for i in range(rep):
        X_train_acc_roll = rolling(X_train_acc_temp)
        #X_train_acc_rp = permutation(rolling(X_train_acc_temp)) # rolling + permutation

        X_train_gy_roll = rolling(X_train_gy_temp)
        #X_train_gy_rp = permutation(rolling(X_train_gy_temp)) # rolling + permutation

        # 증강시킨 데이터 원래 데이터에 추가
        X_train_acc = np.append(X_train_acc, X_train_acc_roll, axis=0)
        #X_train_acc = np.append(X_train_acc, X_train_acc_rp, axis=0)

        X_train_gy = np.append(X_train_gy, X_train_gy_roll, axis=0)
        #X_train_gy = np.append(X_train_gy, X_train_gy_rp, axis=0)

        y_train_total = np.append(y_train_total, y_train, axis=0)
        if i != (rep-1): # 마지막 한 번 제외
            y_train_total = np.append(y_train_total, y_train, axis=0)

    return X_train_acc, X_train_gy, y_train_total 

In [6]:
X_train_acc, X_train_gy, y_train_total = start_augmentation(train, train_labels)

X_train_acc.shape, X_train_gy.shape, y_train_total.shape

100%|██████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:37<00:00, 84.33it/s]


NameError: name 'rolling' is not defined

In [ ]:
# np array 형태를 dataframe 으로 변환
def np_to_df(X_train_acc, X_train_gy):
    acc = [e for sl in X_train_acc for e in sl]
    gy = [e for sl in X_train_gy for e in sl]

    df_report_acc = np.stack(acc, axis = 0)
    df_report_gy = np.stack(gy, axis = 0)

    df_acc = pd.DataFrame(df_report_acc, columns= ['acc_x', 'acc_y', 'acc_z']) 
    df_gy = pd.DataFrame(df_report_gy, columns= ['gy_x', 'gy_y', 'gy_z']) 

    # acc, gy 데이터프레임 병합
    df_aug_result = pd.concat([df_acc, df_gy], axis = 1)
    
    return df_aug_result

In [ ]:
train = np_to_df(X_train_acc, X_train_gy)
train

In [ ]:
# 가속도
train['acc_t'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2) ** (1/3)
test['acc_t'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2) ** (1/3)

train['gy_t'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2) ** (1/3)
test['gy_t'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2) ** (1/3)

# Signal 극대화 (peak 캐치 유용)
train['acc_mag'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2)
test['acc_mag'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2)

train['gy_mag'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2)
test['gy_mag'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2)

In [ ]:
# vector
train['acc_vec'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_y'] ** 2)+(train['acc_z'] ** 2))
test['acc_vec'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_y'] ** 2)+(test['acc_z'] ** 2))

train['gy_vec'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_y'] ** 2)+(train['gy_z'] ** 2))
test['gy_vec'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_y'] ** 2)+(test['gy_z'] ** 2))

# 자이로스코프 무게중심
train['gy_gravity'] = (train['gy_x']+train['gy_y']+train['gy_z'])/3
test['gy_gravity'] = (test['gy_x']+test['gy_y']+test['gy_z'])/3

In [ ]:
# roll & pitch
train['roll'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2))
test['roll'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2))

train['pitch'] = np.arctan(train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2))
test['pitch'] = np.arctan(test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2))

train['math_roll'] = np.arctan(- train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_roll'] = np.arctan(- test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['math_pitch'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_pitch'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['gy_roll'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2))
test['gy_roll'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2))

train['gy_pitch'] = np.arctan(train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2))
test['gy_pitch'] = np.arctan(test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2))

train['gy_math_roll'] = np.arctan(- train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_roll'] = np.arctan(- test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

train['gy_math_pitch'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_pitch'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

print(train.shape)
train

In [13]:
# Scaling 원하는 걸로 사용
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

scaler = RobustScaler()
train = scaler.fit_transform(train)
test.drop(['id', 'time'], axis=1, inplace=True)
test = scaler.transform(test)
train

array([[ 1.93489817,  0.03657354, -0.31330862, ...,  0.05924213,
        -0.05924213, -0.71658616],
       [ 2.01969699,  0.00673528, -0.37289479, ...,  0.07768724,
        -0.07768724, -0.94204715],
       [ 2.0372705 ,  0.01260973, -0.49722803, ...,  0.0059123 ,
        -0.0059123 , -0.98671563],
       ...,
       [-0.34391066,  0.25660796, -0.40865771, ..., -0.27397461,
         0.27397461, -0.13581192],
       [-0.25697945,  0.18887482, -0.38165898, ..., -0.23468183,
         0.23468183,  0.85012356],
       [-0.18474634,  0.13070128, -0.37485685, ..., -0.03601273,
         0.03601273,  1.31550233]])

In [14]:
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dropout, LSTM, Input
import tensorflow_decision_forests as tfdf

from keras.layers import TimeDistributed
from keras.layers import Activation, GlobalAveragePooling1D
from keras.layers import Dense, Flatten, BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from keras.models import load_model
from keras.layers.merge import concatenate

In [15]:
len_features = train.shape[1] # feature 갯수
X = train.reshape(-1, 600, len_features)
X.shape

(21875, 600, 21)

In [16]:
len_features = train.shape[1] # feature 갯수
X = train.reshape(-1, 600, len_features)
X.shapey = to_categorical(y_train_total) 
y.shape

(21875, 61)

In [17]:
epochs, batch_size = 100, 64 # Ram 24GB 기준 256 이상 reset될 가능성 높음

In [18]:
test_X = test.reshape(-1, 600, len_features)
test_X.shape

(782, 600, 21)

In [19]:
#epochs, batch_size = 30, 64
n_features, n_outputs = X.shape[2], y.shape[1]
# reshape data into time steps of sub-sequences
n_steps, n_length = 6, 100
X = X.reshape((X.shape[0], n_steps, n_length, n_features))
test_X = test_X.reshape((test_X.shape[0], n_steps, n_length, n_features))

In [20]:
class Models:
    # 각종 사이즈는 다양하게 적용하여 stacking 쌓아 올리기
    def define_model_0():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
        model.add(TimeDistributed(Flatten()))
        model.add(LSTM(32))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    def define_model_1():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
        model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
        model.add(TimeDistributed(Flatten()))
        model.add(LSTM(32))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    def define_model_2():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=6, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
        model.add(TimeDistributed(Flatten()))
        model.add(LSTM(32))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    def define_model_3():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=6, activation='relu')))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
        model.add(TimeDistributed(Flatten()))
        model.add(LSTM(32))
        model.add(Dropout(0.3))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    def define_model_4():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(GlobalAveragePooling1D()))
        model.add(TimeDistributed(Flatten()))
        model.add(LSTM(32))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

In [21]:
# 위의 모델들 학습하면서 stacking
for i in range(5):
    model = getattr(Models, f'define_model_{i}')()
    checkpoint_path = "checkpoint/cp.ckpt"
    cp_callback = ModelCheckpoint(filepath=checkpoint_path, monitor='loss', 
                                verbose=1, save_weights_only=True, 
                                save_best_only=True, mode='min')
    early_stopping = EarlyStopping(monitor='loss', patience=5, mode='min')
    model.fit(X, y, epochs=epochs, batch_size=batch_size, 
            validation_split=0.2, callbacks=[early_stopping, cp_callback])
    model.save(f'models/211008_model_{i}.h5')
    tf.keras.backend.clear_session()

Epoch 1/100
274/274 [==============================] - 31s 105ms/step - loss: 2.7582 - accuracy: 0.4827 - val_loss: 2.6247 - val_accuracy: 0.4919

Epoch 00001: loss improved from inf to 2.75816, saving model to checkpoint\cp.ckpt
Epoch 2/100
274/274 [==============================] - 28s 103ms/step - loss: 2.4614 - accuracy: 0.4981 - val_loss: 2.5446 - val_accuracy: 0.4917

Epoch 00002: loss improved from 2.75816 to 2.46143, saving model to checkpoint\cp.ckpt
Epoch 3/100
274/274 [==============================] - 26s 96ms/step - loss: 2.3739 - accuracy: 0.4983 - val_loss: 2.5255 - val_accuracy: 0.4969

Epoch 00003: loss improved from 2.46143 to 2.37394, saving model to checkpoint\cp.ckpt
Epoch 4/100
274/274 [==============================] - 27s 97ms/step - loss: 2.3066 - accuracy: 0.5007 - val_loss: 2.4943 - val_accuracy: 0.4910

Epoch 00004: loss improved from 2.37394 to 2.30664, saving model to checkpoint\cp.ckpt
Epoch 5/100
274/274 [==============================] - 26s 95ms/step -

274/274 [==============================] - 26s 96ms/step - loss: 1.6604 - accuracy: 0.5971 - val_loss: 2.1844 - val_accuracy: 0.5422

Epoch 00036: loss did not improve from 1.65737
Epoch 37/100
274/274 [==============================] - 26s 95ms/step - loss: 1.6389 - accuracy: 0.6032 - val_loss: 2.1790 - val_accuracy: 0.5483

Epoch 00037: loss improved from 1.65737 to 1.63891, saving model to checkpoint\cp.ckpt
Epoch 38/100
274/274 [==============================] - 27s 98ms/step - loss: 1.6300 - accuracy: 0.6070 - val_loss: 2.1790 - val_accuracy: 0.5461

Epoch 00038: loss improved from 1.63891 to 1.63001, saving model to checkpoint\cp.ckpt
Epoch 39/100
274/274 [==============================] - 30s 108ms/step - loss: 1.6286 - accuracy: 0.6058 - val_loss: 2.1719 - val_accuracy: 0.5481

Epoch 00039: loss improved from 1.63001 to 1.62864, saving model to checkpoint\cp.ckpt
Epoch 40/100
274/274 [==============================] - 28s 102ms/step - loss: 1.6079 - accuracy: 0.6078 - val_loss:


Epoch 00072: loss improved from 1.40468 to 1.40466, saving model to checkpoint\cp.ckpt
Epoch 73/100
274/274 [==============================] - 26s 95ms/step - loss: 1.4065 - accuracy: 0.6418 - val_loss: 2.3225 - val_accuracy: 0.5477

Epoch 00073: loss did not improve from 1.40466
Epoch 74/100
274/274 [==============================] - 27s 97ms/step - loss: 1.4013 - accuracy: 0.6451 - val_loss: 2.3095 - val_accuracy: 0.5536

Epoch 00074: loss improved from 1.40466 to 1.40134, saving model to checkpoint\cp.ckpt
Epoch 75/100
274/274 [==============================] - 26s 95ms/step - loss: 1.4063 - accuracy: 0.6428 - val_loss: 2.2627 - val_accuracy: 0.5474

Epoch 00075: loss did not improve from 1.40134
Epoch 76/100
274/274 [==============================] - 26s 96ms/step - loss: 1.3969 - accuracy: 0.6425 - val_loss: 2.2713 - val_accuracy: 0.5543

Epoch 00076: loss improved from 1.40134 to 1.39686, saving model to checkpoint\cp.ckpt
Epoch 77/100
274/274 [==============================] - 

274/274 [==============================] - 13s 46ms/step - loss: 2.0186 - accuracy: 0.5369 - val_loss: 2.3486 - val_accuracy: 0.4994

Epoch 00010: loss improved from 2.04729 to 2.01864, saving model to checkpoint\cp.ckpt
Epoch 11/100
274/274 [==============================] - 13s 46ms/step - loss: 1.9672 - accuracy: 0.5463 - val_loss: 2.2819 - val_accuracy: 0.5086

Epoch 00011: loss improved from 2.01864 to 1.96716, saving model to checkpoint\cp.ckpt
Epoch 12/100
274/274 [==============================] - 13s 46ms/step - loss: 1.9367 - accuracy: 0.5518 - val_loss: 2.3255 - val_accuracy: 0.5051

Epoch 00012: loss improved from 1.96716 to 1.93672, saving model to checkpoint\cp.ckpt
Epoch 13/100
274/274 [==============================] - 13s 46ms/step - loss: 1.9093 - accuracy: 0.5549 - val_loss: 2.3376 - val_accuracy: 0.4985

Epoch 00013: loss improved from 1.93672 to 1.90931, saving model to checkpoint\cp.ckpt
Epoch 14/100
274/274 [==============================] - 13s 46ms/step - loss:


Epoch 00045: loss improved from 1.46926 to 1.46811, saving model to checkpoint\cp.ckpt
Epoch 46/100
274/274 [==============================] - 13s 47ms/step - loss: 1.4540 - accuracy: 0.6464 - val_loss: 2.2499 - val_accuracy: 0.5291

Epoch 00046: loss improved from 1.46811 to 1.45404, saving model to checkpoint\cp.ckpt
Epoch 47/100
274/274 [==============================] - 13s 48ms/step - loss: 1.4483 - accuracy: 0.6438 - val_loss: 2.2749 - val_accuracy: 0.5237

Epoch 00047: loss improved from 1.45404 to 1.44832, saving model to checkpoint\cp.ckpt
Epoch 48/100
274/274 [==============================] - 13s 48ms/step - loss: 1.4398 - accuracy: 0.6434 - val_loss: 2.2576 - val_accuracy: 0.5248

Epoch 00048: loss improved from 1.44832 to 1.43983, saving model to checkpoint\cp.ckpt
Epoch 49/100
274/274 [==============================] - 13s 48ms/step - loss: 1.4350 - accuracy: 0.6461 - val_loss: 2.2788 - val_accuracy: 0.5225

Epoch 00049: loss improved from 1.43983 to 1.43498, saving mode

...

274/274 [==============================] - 19s 69ms/step - loss: 1.9919 - accuracy: 0.5515 - val_loss: 2.3008 - val_accuracy: 0.5173

Epoch 00006: loss improved from 2.04844 to 1.99187, saving model to checkpoint\cp.ckpt
Epoch 7/100
274/274 [==============================] - 18s 67ms/step - loss: 1.9508 - accuracy: 0.5617 - val_loss: 2.2538 - val_accuracy: 0.5321

Epoch 00007: loss improved from 1.99187 to 1.95083, saving model to checkpoint\cp.ckpt
Epoch 8/100
274/274 [==============================] - 18s 64ms/step - loss: 1.9112 - accuracy: 0.5663 - val_loss: 2.2422 - val_accuracy: 0.5234

Epoch 00008: loss improved from 1.95083 to 1.91118, saving model to checkpoint\cp.ckpt
Epoch 9/100
274/274 [==============================] - 18s 64ms/step - loss: 1.8725 - accuracy: 0.5754 - val_loss: 2.2070 - val_accuracy: 0.5369

Epoch 00009: loss improved from 1.91118 to 1.87254, saving model to checkpoint\cp.ckpt
Epoch 10/100
274/274 [==============================] - 12s 44ms/step - loss: 1.


Epoch 00041: loss improved from 1.51820 to 1.51120, saving model to checkpoint\cp.ckpt
Epoch 42/100
274/274 [==============================] - 11s 39ms/step - loss: 1.4995 - accuracy: 0.6626 - val_loss: 2.0058 - val_accuracy: 0.5790

Epoch 00042: loss improved from 1.51120 to 1.49949, saving model to checkpoint\cp.ckpt
Epoch 43/100
274/274 [==============================] - 11s 39ms/step - loss: 1.4988 - accuracy: 0.6645 - val_loss: 2.0115 - val_accuracy: 0.5819

Epoch 00043: loss improved from 1.49949 to 1.49879, saving model to checkpoint\cp.ckpt
Epoch 44/100
274/274 [==============================] - 11s 39ms/step - loss: 1.4937 - accuracy: 0.6665 - val_loss: 2.0031 - val_accuracy: 0.5787

Epoch 00044: loss improved from 1.49879 to 1.49374, saving model to checkpoint\cp.ckpt
Epoch 45/100
274/274 [==============================] - 11s 39ms/step - loss: 1.4891 - accuracy: 0.6672 - val_loss: 1.9933 - val_accuracy: 0.5801

Epoch 00045: loss improved from 1.49374 to 1.48909, saving mode

274/274 [==============================] - 11s 41ms/step - loss: 1.3766 - accuracy: 0.6893 - val_loss: 2.0052 - val_accuracy: 0.5861

Epoch 00079: loss did not improve from 1.37190
Epoch 80/100
274/274 [==============================] - 11s 42ms/step - loss: 1.3705 - accuracy: 0.6914 - val_loss: 1.9968 - val_accuracy: 0.5904

Epoch 00080: loss improved from 1.37190 to 1.37054, saving model to checkpoint\cp.ckpt
Epoch 81/100
274/274 [==============================] - 11s 41ms/step - loss: 1.3682 - accuracy: 0.6917 - val_loss: 2.0057 - val_accuracy: 0.5877

Epoch 00081: loss improved from 1.37054 to 1.36823, saving model to checkpoint\cp.ckpt
Epoch 82/100
274/274 [==============================] - 11s 41ms/step - loss: 1.3637 - accuracy: 0.6930 - val_loss: 1.9876 - val_accuracy: 0.5895

Epoch 00082: loss improved from 1.36823 to 1.36367, saving model to checkpoint\cp.ckpt
Epoch 83/100
274/274 [==============================] - 11s 41ms/step - loss: 1.3558 - accuracy: 0.6960 - val_loss: 2

 44/274 [===>..........................] - ETA: 18s - loss: 1.6120 - accuracy: 0.6307

KeyboardInterrupt: 

In [23]:
# 저장한 모델 불러오기
for i in range(5): # 모델 갯수
    globals()[f'model{i}'] = load_model(f'models/211008_model_{i}.h5')

In [25]:
model0._name = 'Client0'
model1._name = 'Client1'
model2._name = 'Client2'
model3._name = 'Client3'
model4._name = 'Client4'

In [26]:
inputs = Input(shape=(n_steps, n_length, n_features))

merge = concatenate([model0(inputs), model1(inputs), model2(inputs), model3(inputs), model4(inputs)])
hidden = Dense(10, activation='relu')(merge)
output = Dense(61, activation='softmax')(hidden)
model = tf.keras.models.Model(inputs=inputs, outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
checkpoint_path = "checkpoint/cp.ckpt"
cp_callback = ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

early_stopping = EarlyStopping(monitor='loss', patience=5, mode='min')
model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.02, callbacks=[early_stopping, cp_callback])

Epoch 1/100
335/335 [==============================] - 120s 329ms/step - loss: 2.9083 - accuracy: 0.4686 - val_loss: 2.9914 - val_accuracy: 0.4863

Epoch 00001: val_loss improved from inf to 2.99139, saving model to checkpoint\cp.ckpt
Epoch 2/100
335/335 [==============================] - 106s 316ms/step - loss: 2.0825 - accuracy: 0.5330 - val_loss: 3.1542 - val_accuracy: 0.4521

Epoch 00002: val_loss did not improve from 2.99139
Epoch 3/100
335/335 [==============================] - 105s 314ms/step - loss: 1.8759 - accuracy: 0.5718 - val_loss: 3.3613 - val_accuracy: 0.4292

Epoch 00003: val_loss did not improve from 2.99139
Epoch 4/100
335/335 [==============================] - 101s 302ms/step - loss: 1.7181 - accuracy: 0.6002 - val_loss: 3.4827 - val_accuracy: 0.4087

Epoch 00004: val_loss did not improve from 2.99139
Epoch 5/100
335/335 [==============================] - 102s 304ms/step - loss: 1.6138 - accuracy: 0.6256 - val_loss: 3.5465 - val_accuracy: 0.4018

Epoch 00005: val_los

335/335 [==============================] - 103s 309ms/step - loss: 1.0285 - accuracy: 0.7382 - val_loss: 4.3462 - val_accuracy: 0.3767

Epoch 00042: val_loss did not improve from 2.99139
Epoch 43/100
335/335 [==============================] - 110s 328ms/step - loss: 1.0396 - accuracy: 0.7374 - val_loss: 4.3618 - val_accuracy: 0.3516

Epoch 00043: val_loss did not improve from 2.99139
Epoch 44/100
335/335 [==============================] - 108s 321ms/step - loss: 1.0354 - accuracy: 0.7377 - val_loss: 4.3094 - val_accuracy: 0.3767

Epoch 00044: val_loss did not improve from 2.99139
Epoch 45/100
335/335 [==============================] - 100s 299ms/step - loss: 1.0262 - accuracy: 0.7399 - val_loss: 4.3382 - val_accuracy: 0.3676

Epoch 00045: val_loss did not improve from 2.99139
Epoch 46/100
335/335 [==============================] - 106s 316ms/step - loss: 1.0213 - accuracy: 0.7377 - val_loss: 4.2336 - val_accuracy: 0.3447

Epoch 00046: val_loss did not improve from 2.99139
Epoch 47/100


In [ ]:
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import accuracy_score

# stfold = StratifiedKFold(n_splits=3, shuffle=True)
# idx_iter = 0 
# skf_accuracy=[]

# for train_idx, valid_idx in stfold.split(X, y_train_total) : 
#     Y_train, Y_valid = tf.gather(y, train_idx), tf.gather(y, valid_idx)
#     X_train, X_valid = tf.gather(X, train_idx), tf.gather(X, valid_idx)

#     checkpoint_path = "checkpoint/cp(211008).ckpt"
#     cp_callback = ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

#     early_stopping = EarlyStopping(monitor='loss', patience=5, mode='min')
#     model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, cp_callback])
#     pred = model.predict(X_valid)

#     # 반복 시 마다 정확도 측정 -> 수정해야함; 
#     idx_iter += 1 
#     y_pred = (pred > 0.5) 
#     accuracy = np.round(accuracy_score(Y_valid, y_pred), 4)
#     train_size = X_train.shape[0]
#     test_size = X_valid.shape[0]

#     print("\n##### 교차 검증: {}, 정확도: {}  #####" .format(idx_iter, accuracy))
#     print('학습 레이블 데이터 분포:\n ', Y_train.shape[0])
#     print('검증 레이블 데이터 분포:\n ', Y_valid.shape[0], '\n\n')

In [50]:
prediction = model.predict(test_X)
prediction.shape

(782, 61)

In [51]:
submission

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,8.188165e-07,3.252405e-09,5.289134e-12,5.288130e-10,1.057025e-04,6.336675e-06,1.272024e-08,1.353531e-08,0.000027,...,3.615741e-08,4.092537e-06,9.319634e-09,1.288084e-05,8.179205e-11,4.108106e-05,0.000020,0.000998,1.578432e-04,2.457487e-07
1,3126,3.557848e-03,3.452285e-05,1.907644e-03,8.422525e-03,8.233026e-04,7.096452e-04,1.453176e-03,3.240900e-04,0.002677,...,3.404642e-04,4.795172e-05,1.079065e-03,1.729587e-03,3.814084e-03,4.433006e-04,0.000325,0.000772,9.722515e-05,2.036020e-03
2,3127,7.310430e-03,3.260876e-02,2.663807e-04,1.955045e-05,4.687956e-07,1.706130e-05,6.450887e-04,1.084616e-02,0.025576,...,7.561062e-05,9.240519e-05,5.300564e-06,7.785617e-03,2.296529e-04,6.304176e-04,0.000013,0.000013,9.044350e-05,1.898947e-03
3,3128,2.403304e-03,2.892588e-06,2.312218e-05,5.341680e-05,3.259653e-05,1.905677e-06,1.216230e-04,3.120205e-05,0.002048,...,1.952361e-06,4.380901e-06,2.154401e-06,4.251190e-05,1.214165e-05,2.761259e-06,0.000412,0.000030,1.425031e-05,1.701997e-03
4,3129,8.270038e-05,7.647411e-09,7.342957e-07,5.336374e-06,5.802468e-05,6.494856e-07,1.975171e-05,4.428545e-07,0.000017,...,3.906394e-08,9.341128e-09,8.720875e-07,5.643861e-07,8.725259e-07,2.958231e-07,0.000267,0.000012,8.937378e-07,4.076185e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,5.334413e-04,2.537272e-07,1.540227e-05,1.291522e-04,1.784978e-03,3.722269e-05,6.304502e-05,1.378330e-05,0.000097,...,1.185205e-06,3.891688e-07,1.386265e-05,4.249470e-06,1.846914e-05,6.107186e-06,0.000760,0.000095,1.536776e-05,2.605067e-04
778,3903,1.417753e-03,9.522214e-06,9.579218e-05,2.781164e-04,1.445989e-04,3.117869e-05,8.242012e-04,3.272770e-05,0.002510,...,1.726760e-05,4.702084e-06,5.571641e-05,1.088487e-04,1.235258e-04,3.478106e-05,0.000856,0.000138,7.609236e-05,7.287843e-04
779,3904,1.166258e-03,1.391959e-06,4.337814e-05,1.262150e-04,7.601050e-05,7.895438e-06,2.302951e-04,2.008697e-05,0.000473,...,3.253240e-06,1.608644e-06,1.338978e-05,1.240644e-04,4.352060e-05,1.083684e-05,0.000348,0.000095,9.161315e-06,7.376492e-04
780,3905,2.303998e-05,6.097020e-03,1.234970e-04,6.617053e-05,2.639443e-06,1.714626e-06,3.114889e-02,1.622851e-06,0.001601,...,2.446040e-06,7.999105e-06,1.727888e-06,1.590365e-03,3.393491e-04,7.269366e-04,0.000080,0.000049,3.950317e-03,2.099279e-05


In [52]:
submission.iloc[:,1:]=prediction

In [53]:
submission.to_csv('cnn_LSTM_stacked5_fold3(jy).csv', index=False)

In [54]:
submission

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,1.818085e-07,2.990748e-10,4.487960e-13,1.291852e-10,1.653824e-04,3.122892e-06,3.212281e-09,4.089149e-09,3.953727e-06,...,5.920189e-09,1.194939e-06,2.049685e-09,2.950898e-06,1.409117e-11,2.571565e-05,1.052109e-05,0.000882,7.647593e-05,4.605368e-08
1,3126,1.035188e-04,7.174468e-08,2.586844e-06,3.508122e-05,1.124479e-04,9.138392e-06,8.007053e-05,3.063066e-06,3.752326e-05,...,3.524654e-07,1.798253e-08,9.352498e-06,3.732722e-06,1.238638e-05,7.568388e-06,1.090419e-04,0.000057,3.633829e-06,1.975011e-05
2,3127,5.698949e-04,1.300783e-02,1.049216e-05,1.284879e-07,2.514458e-09,1.802775e-07,9.249253e-05,1.102775e-03,7.089174e-03,...,2.359332e-05,4.424048e-05,1.849813e-07,1.850916e-03,6.893796e-06,8.602461e-05,8.844395e-07,0.000001,1.894044e-05,3.203505e-04
3,3128,1.129604e-04,8.796709e-09,2.209129e-07,1.394959e-06,3.753279e-06,4.502023e-08,8.851490e-06,4.006568e-07,4.572582e-05,...,5.797814e-09,2.277752e-09,7.398341e-08,1.867475e-07,2.243160e-07,4.317151e-08,7.804685e-05,0.000002,2.850201e-07,1.608517e-05
4,3129,1.273341e-05,2.004103e-10,4.232902e-08,7.597974e-07,9.845483e-06,6.312181e-08,3.018983e-06,5.344403e-08,9.868612e-07,...,7.965164e-10,4.721470e-11,9.768743e-08,2.629502e-08,1.063932e-07,2.587393e-08,3.513558e-05,0.000002,3.781655e-08,1.622565e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,1.589479e-04,1.404485e-08,1.107691e-06,1.604071e-05,3.846888e-04,9.459447e-07,2.214505e-05,1.306076e-06,1.472531e-05,...,2.601518e-08,1.491754e-08,6.096317e-07,3.859403e-07,1.141888e-06,3.284736e-07,6.095179e-04,0.000026,2.337227e-06,6.067682e-05
778,3903,9.217057e-05,4.155473e-08,6.853861e-07,6.811353e-06,5.491576e-05,1.486162e-06,3.743223e-05,1.219919e-06,5.136179e-05,...,5.491746e-08,5.318538e-09,1.334689e-06,3.455058e-07,1.717482e-06,7.994167e-07,2.084754e-04,0.000011,2.866668e-06,1.306722e-05
779,3904,4.075159e-05,1.473930e-09,1.562473e-07,1.337599e-06,2.428704e-06,3.349579e-08,7.848435e-06,1.075496e-07,1.061142e-05,...,5.234783e-09,6.082884e-10,1.357391e-07,1.431384e-07,2.059154e-07,3.035527e-08,4.723554e-05,0.000002,8.771500e-08,8.799493e-06
780,3905,2.302760e-04,2.713558e-02,2.443792e-04,2.005777e-04,3.298060e-05,6.894526e-05,3.003283e-02,1.486704e-04,1.839368e-02,...,4.510919e-04,1.025270e-03,4.981594e-05,9.963953e-03,1.301768e-03,1.893748e-02,2.438040e-04,0.001290,3.344771e-02,3.274780e-04


In [59]:
def result_pred(sub1, sub2) :
    subpre1 = []
    subpre2 = []
    corcount = 0
    
    for i in range(len(sub1)) :
        subpre1.append(sub1.iloc[i,1:].idxmax())
        subpre2.append(sub2.iloc[i,1:].idxmax())

    subpre1 = pd.DataFrame(subpre1, columns = ['label'])
    subpre1.head()
    
    subpre2 = pd.DataFrame(subpre2, columns = ['label'])
    subpre2.head()
    
    print("파일 1 운동중 라벨 수 :", len(subpre1[subpre1['label'] != '26']), "\n파일 2 운동중 라벨 수 :", len(subpre2[subpre2['label'] != '26']))
    for i in range(len(subpre1)) :
        if (subpre1['label'][i] == subpre2['label'][i]) : 
            corcount += 1
    print("예측 결과 일치 개수 :", corcount)

In [63]:
sub1 = pd.read_csv('cnn_LSTM_stacked5_fold3(jy).csv')
sub2 = pd.read_csv('cnn_LSTM_stacked_5(jy).csv')

result_pred(sub1, sub2)

파일 1 운동중 라벨 수 : 780 
파일 2 운동중 라벨 수 : 380
예측 결과 일치 개수 : 1


In [ ]:
sub1

In [ ]:
sub2